In [ ]:
import Pkg
pkgs = [
"DataFrames",
"Dates",
"uCSV",
"Revise",
"StatsBase",
"StatsPlots"
]
for pkg in pkgs
    eval(Meta.parse("import $pkg"))
end
import Mycelia

In [ ]:
data_dir = joinpath(dirname(pwd()), "data")

In [ ]:
gzipped_fasta = joinpath(data_dir, "results/high-confidence-hiv-contigs.fna.gz")
fasta = replace(gzipped_fasta, ".gz" => "")
run(pipeline(`gzip -dc $(gzipped_fasta)`, fasta))
run(`conda run --no-capture-output -n prodigal prodigal -m -p meta -i $(fasta) -f gff -o $(fasta).prodigal.gff -a $(fasta).prodigal.faa -d $(fasta).prodigal.fna -s $(fasta).prodigal.all_potential_gene_scores.txt`)

In [ ]:
target_database = "$(homedir())/workspace/mmseqs/UniRef100"
# target_database = "$(homedir())/workspace/mmseqs/UniRef90"
# target_database = "$(homedir())/workspace/mmseqs/UniRef50"
outfile_base = "$(fasta).prodigal.faa.mmseqs_easy_search." * basename(target_database)
# run(`sbatch $(homedir())/workspace/Mycelia/scripts/mmseqs-easy-search.sh $(fasta).prodigal.faa $target_database $outfile_base`)

In [ ]:
# mmseqs_outfile = Mycelia.run_mmseqs_easy_search(out_dir=OUT_DIR, query_fasta=amino_acid_fasta, target_database="$(homedir())/workspace/mmseqs/$(target_database)", outfile=identifier)
# updated_gff_table = Mycelia.update_gff_with_mmseqs("$(fasta).prodigal.gff", outfile_base)
# updated_gff_file = "$(fasta).prodigal.mmseqs$(basename(target_database)).gff"
# uCSV.write(updated_gff_file, data = collect(DataFrames.eachcol(updated_gff_table)), header=missing, delim='\t')
# run(`seqret -sequence $(fasta) -feature -fformat gff -fopenfile $(updated_gff_file) -osformat genbank -osname_outseq $(updated_gff_file) -ofdirectory_outseq gbk_file -auto`)

In [ ]:
mmseqs_results = DataFrames.DataFrame(uCSV.read(outfile_base, typedetectrows=100, delim='\t', header=1))
mmseqs_results = mmseqs_results[map(x -> occursin(r"vir"i, x), mmseqs_results[!, "theader"]), :]
sorted_hits = sort(collect(StatsBase.countmap(mmseqs_results[!, "query"])), by=x->x[2], rev=true)
StatsPlots.scatter(
    mmseqs_results[!, "alnlen"],
    mmseqs_results[!, "pident"]
)

In [ ]:
# sort(mmseqs_results, "alnlen", rev=true)

In [ ]:
# mmseqs_results[!, "alnlen"]

In [ ]:
# for sorted_hit in sorted_hits
#     if sorted_hit[2] >= 100
#         println(sorted_hit[1])
#         for x in mmseqs_results[mmseqs_results[!, "query"] .== sorted_hit[1], "theader"]
#             println(x)
#         end
#     end
# end